In [2]:
import praw
import pandas as pd
import datetime
import concurrent.futures

reddit = praw.Reddit(client_id='_37znztIqSaRtA8ZWOCRXg', client_secret="0wz5rUMlMjJBSCwUuXhuDt_64OOCyA",
                     user_agent='CIS509')

subreddit = reddit.subreddit('UberEats')

def process_submission(submission):
    try:
        submission.comments.replace_more(limit=0)
        comments_data = []
        for comment in submission.comments.list():
            timestamp = datetime.datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
            comments_data.append([comment.id,
                                 comment.author,
                                 timestamp,
                                 comment.parent_id,
                                 submission.id,
                                 submission.title,
                                 comment.body,
                                 comment.permalink])
        return comments_data
    except Exception as e:
        print(f"Error processing submission with id {submission.id}: {e}")
        return []

comments_data = []
count = 0
with concurrent.futures.ThreadPoolExecutor() as executor:
    submission_futures = [executor.submit(process_submission, submission) for submission in subreddit.new(limit=1000)]
    for submission_future in concurrent.futures.as_completed(submission_futures):
        comments_data += submission_future.result()
        count += len(submission_future.result())
        print(f"Processed {count} comments.")
        if count >= 2000:
            break

comments_df = pd.DataFrame(comments_data, columns=['Comment ID', 'Author', 'Timestamp',
                                                    'Comment is a reply to', 'Comment Thread ID',
                                                    'Thread Title', 'Comment Body', 'Comment Permalink'])

print(comments_df)


Processed 45 comments.
Processed 51 comments.
Processed 51 comments.
Processed 57 comments.
Processed 74 comments.
Processed 76 comments.
Processed 76 comments.
Processed 80 comments.
Processed 87 comments.
Processed 97 comments.
Processed 113 comments.
Processed 113 comments.
Processed 117 comments.
Processed 122 comments.
Processed 135 comments.
Processed 140 comments.
Processed 164 comments.
Processed 184 comments.
Processed 192 comments.
Processed 304 comments.
Processed 309 comments.
Processed 313 comments.
Processed 319 comments.
Processed 330 comments.
Processed 369 comments.
Processed 369 comments.
Processed 378 comments.
Processed 387 comments.
Processed 392 comments.
Processed 396 comments.
Processed 535 comments.
Processed 546 comments.
Processed 566 comments.
Processed 566 comments.
Processed 577 comments.
Processed 579 comments.
Processed 583 comments.
Processed 588 comments.
Processed 612 comments.
Processed 624 comments.
Processed 627 comments.
Processed 627 comments.
Pr

In [3]:
comments_df.to_csv('UberEats_comments_21_03.csv',index=False)

In [4]:
!pip install textblob

In [5]:
!pip install nltk

In [6]:
!pip install pandas

In [7]:
import pandas as pd
data = pd.read_csv('UberEats_comments_21_03.csv')
data.head(2)

,Comment ID,Author,Timestamp,Comment is a reply to,Comment Thread ID,Thread Title,Comment Body,Comment Permalink
0,jd0njqz,jack2ho,2023-03-20 16:46:16,t3_11x0892,11x0892,How much do you make from Uber Eats with E-Bike,Bike mode is dead. I'm in Vancouver.,/r/UberEATS/comments/11x0892/how_much_do_you_m...
1,jd0p20u,gtaeast91,2023-03-20 16:57:02,t3_11x0892,11x0892,How much do you make from Uber Eats with E-Bike,The common response is that people are making ...,/r/UberEATS/comments/11x0892/how_much_do_you_m...


In [8]:
#Parse csv's to only extract column called 'message'
messages = data['Comment Body']

In [9]:
from textblob import TextBlob
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
#Text pre-processing functions
def text_processing(message):
    
    #Generating the list of words in the message (hastags and other punctuations removed) and convert to lowercase
    def form_sentence(message):
        message = message.lower() #Make messages lowercase 
        message_blob = TextBlob(message.lower()) #Convert to 'textblob' which provides a simple API for NLP tasks
        return ' '.join(message_blob.words)
    new_message = form_sentence(message)
    
    #Removing stopwords and words with unusual symbols
    def no_user_alpha(message):
        message_list = [item for item in message.split()] 
        clean_words = [word for word in message_list if re.match(r'[^\W\d]*$', word)] #remove punctuation and strange characters
        clean_sentence = ' '.join(clean_words) 
        clean_mess = [stopword for stopword in clean_sentence.split() if stopword not in stopwords.words('english')] #remove stopwords
        return clean_mess
    no_punc_message = no_user_alpha(new_message)
    
    #Normalizing the words in messages 
    def normalization(message_list):
        lem = WordNetLemmatizer()
        normalized_message = []
        for word in message_list:
            normalized_text = lem.lemmatize(word,'v') #lemmatize words
            normalized_message.append(normalized_text)
        return normalized_message
    return normalization(no_punc_message)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sirik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sirik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sirik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sirik\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [10]:
#Print to console and write to file
f = open('processed_text_ubereats.csv','w', encoding='utf8')
for message in messages: 
	message = text_processing(message)
	for term in message:
		f.write(term+" ")
	f.write("\n")

In [11]:
f.close()

In [12]:
!pip install spacy

In [13]:
!python -m spacy download en_core_web_lg

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [14]:
!pip install pydantic --upgrade

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\sirik\\anaconda3\\Lib\\site-packages\\~ydantic\\annotated_types.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.8.2
    Uninstalling pydantic-1.8.2:
      Successfully uninstalled pydantic-1.8.2


In [15]:
!pip install blis

In [16]:
!pip install thinc

  Using cached pydantic-1.8.2-cp39-cp39-win_amd64.whl (1.9 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.6
    Uninstalling pydantic-1.10.6:
      Successfully uninstalled pydantic-1.10.6


In [17]:
!pip install typing_extensions>=3.7.4

In [18]:
import spacy
nlp = spacy.load("en_core_web_lg")
docs = list(nlp.pipe(data['Comment Body']))

In [19]:
import operator
np_dict = {}
for doc in docs:
    for np in doc.noun_chunks:
        np_dict[np.text]=np_dict.get(np.text,0)+1
sortedDict = sorted(np_dict.items(),key=operator.itemgetter(1),reverse=True)
print(sortedDict)

[('I', 1823), ('it', 955), ('you', 821), ('they', 410), ('that', 337), ('me', 295), ('It', 183), ('them', 165), ('this', 146), ('i', 142), ('what', 133), ('You', 113), ('They', 104), ('That', 101), ('we', 82), ('people', 65), ('the order', 60), ('the customer', 60), ('Uber', 59), ('who', 55), ('all', 52), ('This', 51), ('he', 50), ('the food', 50), ('the app', 48), ('which', 46), ('something', 46), ('someone', 43), ('anything', 38), ('support', 38), ('the tip', 37), ('the restaurant', 33), ('food', 32), ('she', 31), ('orders', 30), ('us', 30), ('UE', 28), ('What', 28), ('drivers', 27), ('a lot', 27), ('an order', 24), ('nothing', 22), ('time', 22), ('the time', 21), ('the driver', 20), ('the end', 18), ('customers', 18), ('money', 17), ('door', 16), ('my car', 15), ('my area', 15), ('tip', 15), ('those', 15), ('yourself', 14), ('some', 14), ('everything', 14), ('a supervisor', 14), ('We', 14), ('him', 14), ('the door', 14), ('the day', 13), ('car', 13), ('shop', 13), ('order', 13), ('a

In [20]:
entity_dict = {}
for doc in docs:
    for entity in doc.ents:
        entity_dict[entity.text]=entity_dict.get(entity.text,0)+1
sortedDict = sorted(entity_dict.items(),key=operator.itemgetter(1),reverse=True)
print(sortedDict)

[('2', 41), ('first', 36), ('two', 35), ('one', 32), ('3', 30), ('today', 26), ('Uber', 26), ('🤣', 24), ('second', 21), ('1', 21), ('5', 20), ('10', 18), ('4', 18), ('UE', 17), ('three', 14), ('8', 14), ('yesterday', 14), ('22', 12), ('15', 12), ('tonight', 12), ('LA', 12), ('Instacart', 11), ('100%', 11), ('100', 10), ('6', 10), ('all day', 9), ('Walmart', 9), ('400', 9), ('half', 8), ('10 minutes', 8), ('50', 8), ('this week', 8), ('Red Bull', 8), ('Toronto', 7), ('150', 7), ('California', 7), ('One', 7), ('20', 7), ('an hour', 7), ('Santa Monica', 7), ('the day', 6), ('Panera', 6), ('25', 6), ('14', 6), ('Sunday', 6), ('7', 6), ('40', 6), ('20 minutes', 6), ('a mile', 6), ('12', 6), ('Prop 22', 6), ('UberEats', 6), ('Alani', 6), ('Vancouver', 5), ('Today', 5), ('Doordash', 5), ('30', 5), ('about 10 minutes', 5), ('NYC', 5), ('Reddit', 5), ('2 hours', 5), ('hours', 4), ('a week', 4), ('a month', 4), ('200', 4), ('0', 4), ('3 hours', 4), ('2 miles', 4), ('Atleast', 4), ('the end of th

In [21]:
# To get SpaCy tag list :https://machinelearningknowledge.ai/tutorial-on-spacy-part-of-speech-pos-tagging/

#function to get most common verbs
from collections import Counter
verbs = []
for doc in docs:
    for token in doc: 
        if (not token.is_stop and not token.is_punct and token.pos_=="VERB"):
            verbs.append(token.lemma_)
verb_freq = Counter(verbs)
common_verbs = verb_freq.most_common(10)
print(common_verbs)

[('get', 181), ('think', 146), ('know', 137), ('go', 136), ('pay', 123), ('say', 118), ('work', 106), ('try', 106), ('take', 97), ('deliver', 90)]
